In [ ]:
import torch
import math

In [ ]:
def subdivide(pos, div):
    torch.linspace([1,0],[0,1])

pos = torch.tensor([[1,0],[0,1],[-1,0],[0,-1]])


_ = plt.scatter(pos[:,0], pos[:,1])

In [ ]:
beta = torch.linspace(0, 2*math.pi,2)
gamma = torch.linspace(-math.pi/2, math.pi/2, 2)

pos = torch.stack((torch.cos(a), torch.sin(a)))



_ = plt.scatter(pos[0], pos[1])

In [ ]:


def RotationMatrix(theta, axis):
    N = theta.size(0)
    R = torch.zeros(N, 9)
    
    if axis == 0:
        # [[1, 0, 0], [0, c, -s], [0, s, c]]
        R[:, 4] = torch.cos(theta)
        R[:, 5] = -torch.sin(theta)
        R[:, 7] = torch.sin(theta)
        R[:, 8] = torch.cos(theta)
        R[:, 0] = 1.
    if axis == 1:
        # [[c, 0, s], [0, 1, 0], [-s, 0, c]]
        R[:, 0] = torch.cos(theta)
        R[:, 2] = torch.sin(theta)
        R[:, 6] = -torch.sin(theta)
        R[:, 8] = torch.cos(theta)
        R[:, 4] = 1.
    if axis == 2:
        # [[c, -s, 0], [s, c, 0], [0, 0, 1]]
        R[:, 0] = torch.cos(theta)
        R[:, 1] = -torch.sin(theta)
        R[:, 3] = torch.sin(theta)
        R[:, 4] = torch.cos(theta)
        R[:, 8] = 1.

    return R.reshape(N, 3, 3)

In [ ]:
nalpha = 5
nbeta = 5
ngamma = 5

beta = torch.arange(0.0, math.pi, math.pi / nbeta)
gamma = torch.arange(0.0, 2*math.pi, 2*math.pi / ngamma)

beta, gamma

In [ ]:
Beta = RotationMatrix(beta, 1)
Gamma = RotationMatrix(gamma, 2)

In [ ]:
All = (Gamma.unsqueeze(0) @ Beta.unsqueeze(1)).reshape(-1, 3, 3)

In [ ]:
origin = torch.tensor([1., 0., 0.])

In [ ]:
pos = All @ origin
pos

In [ ]:
fig = plt.figure(figsize=(8.0, 8.0))

ax = fig.add_subplot(
    111,
    projection="3d",
    xlim=(-1., 1.),
    ylim=(-1., 1.),
    zlim=(-1., 1.)
)

im = ax.scatter(
    pos[:,0],
    pos[:,1],
    pos[:,2]
)

In [ ]:
s = torch.rand(10000, 3)*2 - 1

In [ ]:
mask = s.pow(2).sum(dim=1) <= 1

In [ ]:
def norm(tensor):
    return tensor/(tensor.pow(2).sum(dim=1).unsqueeze(1))

In [ ]:
fig = plt.figure(figsize=(8.0, 8.0))

ax = fig.add_subplot(
    111,
    projection="3d",
    xlim=(-1., 1.),
    ylim=(-1., 1.),
    zlim=(-1., 1.)
)

im = ax.scatter(
    norm(s[mask])[:784, 0],
    norm(s[mask])[:784, 1],
    norm(s[mask])[:784, 2]
)

In [ ]:
def sub_divide(vertices, faces, num_iter):
    vertices /= torch.norm(vertices, dim=1, keepdim=True)
    for _ in range(num_iter):
        sub_faces = torch.stack((faces[:, [0, 1, 2]], faces[:, [2, 0, 1]], faces[:, [1, 2, 0]]), dim=1)
        vertices_1 = vertices[sub_faces[:, :, [0, 1]]].mean(dim=2)
        vertices_1 /= torch.norm(vertices_1, dim=2, keepdim=True)
        vertices_2 = vertices[sub_faces[:, :, [1, 2]]].mean(dim=2)
        vertices_2 /= torch.norm(vertices_2, dim=2, keepdim=True)
        
        N = vertices.shape[0]
        NV1 = vertices_1.shape[0]*vertices_1.shape[1]
        NV2 = vertices_2.shape[0]*vertices_1.shape[1]
        
        sub_faces[:,:,0] = torch.arange(N, N+NV1).reshape(-1, 3)
        sub_faces[:,:,2] = torch.arange(N+NV1, N+NV1+NV2).reshape(-1, 3)
        
        faces = torch.cat((sub_faces, torch.arange(N, N+NV1).reshape(-1,1, 3)), dim=1).reshape(-1, 3)
        vertices = torch.cat((vertices, vertices_1.reshape(-1, 3), vertices_2.reshape(-1, 3)), dim=0)
        
    return vertices.unique(dim=0)

In [ ]:
phi = (1 + math.sqrt(5))/2

icosahedron_vertices = torch.tensor([[phi, 1., 0.], [-phi, 1., 0.], [phi, -1., 0.], [-phi, -1., 0.],
                                     [1., 0., phi], [1., 0., -phi], [-1., 0., phi], [-1., 0., -phi],
                                     [0., phi, 1.], [0., -phi, 1.], [0., phi, -1.], [0., -phi, -1.]])
icosahedron_faces = torch.tensor([[0, 2, 4], [0, 2, 5], [0, 4, 8], [0, 8, 10], [0, 5, 10],
                                  [1, 3, 6], [1, 3, 7], [1, 6, 8], [1, 7, 10], [1, 8, 10],
                                  [2, 4, 9], [2, 5, 11], [2, 9, 11], [3, 7, 11], [3, 9, 11],
                                  [3, 6, 9], [4, 6, 9], [4, 6, 8], [5, 7, 10], [5, 7, 11]])

In [ ]:
octahedron_vertices = torch.tensor([[1., 0., 0.], [0., 1., 0.], [-1., 0., 0.], 
                                    [0., -1., 0.], [0., 0., 1.], [0., 0., -1.]])
octahedron_faces = torch.tensor([[0, 1, 5], [1, 5, 2], [2, 5, 3], [0, 5, 3], 
                                 [0, 3, 4], [0, 1, 4], [1, 2, 4], [2, 3, 4]])

In [ ]:
vertices_ = sub_divide(icosahedron_vertices, icosahedron_faces, 5)
vertices_.shape

In [ ]:
fig = plt.figure(figsize=(8.0, 8.0))

ax = fig.add_subplot(
    111,
    projection="3d",
    xlim=(-1., 1.),
    ylim=(-1., 1.),
    zlim=(-1., 1.)
)

im = ax.scatter(
    vertices_[:, 0],
    vertices_[:, 1],
    vertices_[:, 2]
)

In [ ]:
vertices_.shape

In [ ]:
def inv_stereographic_proj(X):
    X_norm = X.pow(2).sum(dim=1).unsqueeze(1)
    x = torch.cat((2*X/(X_norm + 1), (X_norm - 1)/(X_norm + 1)), dim=1)
    return x  

In [ ]:
a = torch.arange(-10, 10).unsqueeze(1)
a

In [ ]:
import matplotlib.pyplot as plt

proj = inv_stereographic_proj(a)
print(proj.shape)
plt.scatter(proj[:,0], proj[:,1])

In [ ]:
torch.cat((torch.rand((10, 2)), torch.rand((10, 1))), dim=1)